# Training Magnification Model

### Setting

In [1]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

video_path = "video/speed80.mp4"
output_dir = "data/frames_sample/"

### Cut Video into Image for Each Frame

In [2]:
from creating_dataset import *

extract_frames_from_video(video_path, output_dir, prefix="sample", every_nth=1)

Using cuda device


Extracting frames from video/speed80.mp4: 100%|██████████| 3630/3630 [05:06<00:00, 11.83it/s]

Saved 3630 frames to: data/frames_sample/


### Creating Xa and Xb Image pairs

In [ ]:
frame_dir = "data/frames_sample"
xa_dir = "data/training_data/L/Xa"
xb_dir = "data/training_data/L/Xb"

prepare_xa_xb_pairs(frame_dir, xa_dir, xb_dir, step=1)

Creating Xa/Xb pairs:  26%|██▌       | 948/3629 [18:08<49:43,  1.11s/it]  

### Creating Magnified Y Data within Xa and Xb

In [4]:
generate_Y_with_optical_flow(xa_dir, xb_dir, "data/training_data/L/Y", sf=50.0)

Generating Y with Optical Flow: 100%|██████████| 3629/3629 [2:25:03<00:00,  2.40s/it]  

Saved 3629 warped Y images using Optical Flow to data/training_data/L/Y


### Model Structure

In [6]:
from magnification_model import *

model = MagnificationModel().to(device)

input_Xa = torch.randn(1, 3, 224, 224).to(device)
input_Xb = torch.randn(1, 3, 224, 224).to(device)
SF = torch.tensor(20.0).to(device)

summary(model, input_data=(input_Xa, input_Xb, SF), col_names=["input_size", "output_size", "num_params"], depth=3)

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
MagnificationModel                       [1, 3, 224, 224]          [1, 3, 224, 224]          --
├─Encoder: 1-1                           [1, 3, 224, 224]          [1, 32, 112, 112]         --
│    └─Sequential: 2-1                   [1, 3, 224, 224]          [1, 32, 112, 112]         --
│    │    └─Conv2d: 3-1                  [1, 3, 224, 224]          [1, 16, 224, 224]         2,368
│    │    └─ReLU: 3-2                    [1, 16, 224, 224]         [1, 16, 224, 224]         --
│    │    └─Conv2d: 3-3                  [1, 16, 224, 224]         [1, 32, 112, 112]         4,640
│    │    └─ReLU: 3-4                    [1, 32, 112, 112]         [1, 32, 112, 112]         --
│    └─Sequential: 2-2                   [1, 32, 112, 112]         [1, 32, 112, 112]         --
│    │    └─ResidualBlock: 3-5           [1, 32, 112, 112]         [1, 32, 112, 112]         18,496
│    │    └─ResidualBlock

### Train
Load the model with `MagnificationModel()`
Load the device with cuda if it is possible

`train()` variable:
- model: loaded magnification model
- Xa: input image at index i
- Xb: input image at index i+1
- Y: magnified image(GT)
- device: cuda or cpu
- num_epochs: number of epoch
- batch_size: size of batch
- save_path: the path to save the model


In [ ]:
from training import train

model = MagnificationModel()
device = "cuda" if torch.cuda.is_available() else "cpu"

train(
    model=model,
    xa_dir="data/training_data/L/Xa",
    xb_dir="data/training_data/L/Xb",
    y_dir="data/training_data/L/Y",
    device=device,
    num_epochs=15,
    batch_size=4,
    save_path="model/best_magnification_model_L.pth"
)
